# Lab3: Memory Analysis Lab Deeper Dive

Identifying memory archive with suspicious process with additional detail. Notebook run on a [WIN-FOR](https://github.com/digitalsleuth/WIN-FOR) configured machine.

In [42]:
import os
import re
import time
import math

print(f"Jupyter Lab running in directory {os.getcwd()}")

Jupyter Lab running in directory C:\Users\vboxuser\repos\CYBS-5014\lab3


In [39]:
# Setup

IMG = Path(r"..\iveep\CTICOE-iveep_374_2025-03-17T17-27Z.mem")
CASE_DIR  = Path.cwd() / "output"
VOL = Path(r"C:\Users\vboxuser\repos\volatility3\vol.py")
VOL2 = Path(r"C:\Users\vboxuser\repos\volatility\vol.py")

CREATE_NO_WINDOW = 0x08000000
si = subprocess.STARTUPINFO()
if hasattr(subprocess, "STARTF_USESHOWWINDOW"):
    si.dwFlags |= subprocess.STARTF_USESHOWWINDOW

# Volatility Wrapper Functions

In [47]:
from pathlib import Path
import subprocess, hashlib, json, textwrap, os, sys, shlex
import pefile

def vol(plugin, *args):
    cmd = [sys.executable, str(VOL), "-f", str(IMG), str(plugin)] + [str(a) for a in args]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    if p.returncode != 0:
        sys.stderr.write(err)
    return out

def vol2(plugin, *args):
    cmd = [sys.executable, str(VOL2), "-f", str(IMG), str(plugin)] + [str(a) for a in args]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    if p.returncode != 0:
        sys.stderr.write(err)
    return out

def run_cmd(cmd, live=True, timeout=None):
    print(f"$ {cmd}")
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate(timeout=timeout)
    sout, serr = out.decode(errors="ignore"), err.decode(errors="ignore")
    if live:
        if sout: print(sout)
        if serr: print(serr, file=sys.stderr)
    return p.returncode, sout, serr

def run_headless(argv, timeout=None):
    """Return (rc, stdout, stderr) for external tool calls, headless."""
    p = subprocess.run(
        argv,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=False,
        creationflags=CREATE_NO_WINDOW,
        startupinfo=si,
        timeout=timeout,
    )
    so = p.stdout.decode(errors="ignore")
    se = p.stderr.decode(errors="ignore")
    return p.returncode, so, se

def tool_exists(path): 
    return Path(path).exists() if path else False

def save(txt, name):
    out_path = os.path.join(str(CASE_DIR), str(name))
    if not os.path.isdir(str(CASE_DIR)):
        try:
            os.makedirs(str(CASE_DIR))
        except OSError:
            pass  # already exists
    with open(out_path, 'wb') as f:
        f.write(txt)
    return out_path

from __future__ import print_function

def printvol(txt):
    return print(txt.decode('utf-8'))

def sha256(p):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h.update(chunk)
    return h.hexdigest()

def ascii_strings(blob, minlen=6):
    out = []
    cur = []
    for b in blob:
        if 32 <= b <= 126:
            cur.append(b)
        else:
            if len(cur) >= minlen:
                out.append(bytes(cur).decode(errors="ignore"))
            cur = []
    if len(cur) >= minlen:
        out.append(bytes(cur).decode(errors="ignore"))
    return out

def entropy(data: bytes) -> float:
    if not data: return 0.0
    from math import log2
    freq = [0]*256
    for b in data: freq[b]+=1
    probs = [f/len(data) for f in freq if f]
    return -sum(p*math.log2(p) for p in probs)

## Overview from lab2

- System time in image: **2025‑03‑17 17:27:09 UTC**  
- Kernel: Windows 10 (x64), kernel family **19041**, 4 logical CPUs  
- Prior triage suspects (from Labs 1–2):  
  - `PSEXESVC.exe` (PID 11296) -> `excel-update.exe` (PID 4672) -> `cmd.exe` (PID 13752) -> `powershell.exe` (PID 13156)  
  - **Rogue `spoolsv.exe` (PID 14536)** spawned by interactive PowerShell (Session 2)  
  - `ScreenConnect.*` service/client processes (PIDs 6988, 14756, 1772)

**New Lab 3 goal:** choose one additional suspicious process and deep‑dive it: the **rogue `spoolsv.exe` (PID 14536)** and/or `ScreenConnect` service (PID 6988). since we already looked at the excel-update.exe chain primarily in the last lab.

For this notebook, I will focus on `spoolsv.exe`

# Deep dive into `spoolsv.exe` (PID 14536)

In [21]:
# Identity, command line, genealogy 
!python "{VOL}" -f "{IMG}" windows.pslist --pid 14536
!python "{VOL}" -f "{IMG}" windows.cmdline --pid 14536
!python "{VOL}" -f "{IMG}" windows.pstree | findstr /i 14536

Volatility 3 Framework 2.26.2

PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

14536	15172	spoolsv.exe	0x840e91a11080	9	-	2	False	2025-03-17 17:12:10.000000 UTC	N/A	Disabled



Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


Volatility 3 Framework 2.26.2

PID	Process	Args

14536	spoolsv.exe	C:\Windows\System32\spoolsv.exe



Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


**** 14536	15172	spoolsv.exe	0x840e91a11080	9	-	2	False	2025-03-17 17:12:10.000000 UTC	N/A	\Device\HarddiskVolume3\Windows\System32\spoolsv.exe	C:\Windows\System32\spoolsv.exe	C:\Windows\System32\spoolsv.exe



Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


# Analysis

## Identity & Genealogy

- Parent: `powershell.exe` (PID 15172) in Session 2 (interactive)
- Create time: 2025-03-17 17:12:10 UTC
- Path/args: `C:\Windows\System32\spoolsv.exe`, no arguments

## Notes

The real print spooler service should run as a service under session 0 under the service control tree, not as a child of interactive PowerShell / Session 2. Additionally, the timing aligns with the ealier PsExec/excel-update.exe activity window. This is likely a spoolsv look-alike spawned for post-exploit activity.

In [23]:
# Modules & memory regions
!python "{VOL}" -f "{IMG}" windows.dlllist --pid 14536
!python "{VOL}" -f "{IMG}" windows.vadinfo --pid 14536
!python "{VOL}" -f "{IMG}" windows.malfind --pid 14536

Volatility 3 Framework 2.26.2

PID	Process	Base	Size	Name	Path	LoadTime	File output

14536	spoolsv.exe	0x7ff7eb690000	0xd2000	spoolsv.exe	C:\Windows\System32\spoolsv.exe	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb50170000	0x1f8000	ntdll.dll	C:\Windows\SYSTEM32\ntdll.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4fac0000	0xbd000	KERNEL32.DLL	C:\Windows\System32\KERNEL32.DLL	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4dd20000	0x2f6000	KERNELBASE.dll	C:\Windows\System32\KERNELBASE.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4f180000	0x19f000	USER32.dll	C:\Windows\System32\USER32.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4d990000	0x22000	win32u.dll	C:\Windows\System32\win32u.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4e570000	0x2b000	GDI32.dll	C:\Windows\System32\GDI32.dll	2025-03-17 17:12:10.000000 UTC	Disabled
14536	spoolsv.exe	0x7ffb4daa0000	0x117000


Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


Volatility 3 Framework 2.26.2

PID	Process	Offset	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	Parent	File	File output

14536	spoolsv.exe	0xffff840ea808ed80	0x7ffee000	0x7ffeefff	VadS	PAGE_READONLY	1	1	0x0	N/A	Disabled
14536	spoolsv.exe	0xffff840eab4a9da0	0x3020000	0x302ffff	VadS	PAGE_READWRITE	15	1	0xffff840ea808ed80	N/A	Disabled
14536	spoolsv.exe	0xffff840e9e7c9080	0x2840000	0x2840fff	Vad 	PAGE_READONLY	0	0	0xffff840eab4a9da0	\Windows\System32\en-US\spoolsv.exe.mui	Disabled
14536	spoolsv.exe	0xffff840eab4a9350	0xb70000	0xb70fff	VadS	PAGE_EXECUTE_READWRITE	1	1	0xffff840e9e7c9080	N/A	Disabled
14536	spoolsv.exe	0xffff840ea808e740	0xa40000	0xb3ffff	VadS	PAGE_READWRITE	15	1	0xffff840eab4a9350	N/A	Disabled
14536	spoolsv.exe	0xffff840e98d44b60	0xa00000	0xa0ffff	Vad 	PAGE_READWRITE	0	0	0xffff840ea808e740	N/A	Disabled
14536	spoolsv.exe	0xffff840ea808edd0	0x800000	0x9fffff	VadS	PAGE_READWRITE	19	1	0xffff840e98d44b60	N/A	Disabled
14536	spoolsv.exe	0xffff840e9cfb31a0	0xa20000	0xa3


Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


Volatility 3 Framework 2.26.2

PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm

14536	spoolsv.exe	0xb70000	0xb70fff	VadS	PAGE_EXECUTE_READWRITE	1	1	Disabled	N/A	
fc 48 83 e4 f0 e8 cc 00 00 00 41 51 41 50 52 48 .H........AQAPRH
31 d2 65 48 8b 52 60 48 8b 52 18 51 48 8b 52 20 1.eH.R`H.R.QH.R 
56 48 0f b7 4a 4a 4d 31 c9 48 8b 72 50 48 31 c0 VH..JJM1.H.rPH1.
ac 3c 61 7c 02 2c 20 41 c1 c9 0d 41 01 c1 e2 ed .<a|., A...A....	fc 48 83 e4 f0 e8 cc 00 00 00 41 51 41 50 52 48 31 d2 65 48 8b 52 60 48 8b 52 18 51 48 8b 52 20 56 48 0f b7 4a 4a 4d 31 c9 48 8b 72 50 48 31 c0 ac 3c 61 7c 02 2c 20 41 c1 c9 0d 41 01 c1 e2 ed
14536	spoolsv.exe	0x3e00000	0x48e5fff	VadS	PAGE_EXECUTE_READWRITE	2790	1	Disabled	N/A	
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 ................
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 ................
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 ................
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00


Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        
C:\Users\vboxuser\repos\volatility3\volatility3\framework\deprecation.py:28: FutureWarning: This API (volatility3.plugins.windows.malware.malfind.Malfind.run) will be removed in the first release after 2026-06-07. This plugin has been renamed, please call volatility3.plugins.windows.malware.malfind.Malfind rather than volatility3.plugins.windows.malfind.Malfind.
  warnings.warn(
C:\Users\vboxuser\repos\volatility3\volatility3\framework\deprecation.py:105: FutureWarning: This plugin (volatilit

# Analysis

## Loaded modules

- Network stack/UI libraries present: `winhttp.dll`, `wininet.dll`, `webio.dll`, `ws2_32.dll`, `dnsapi.dll`, `mswsock.dll`
- Script/.NET/AMSI chain: `mscoree.dll` (.NET CLR host), `amsi.dll` (Antimalware scan interface), `wldp.dll` (Code integrity/WDAC)
- Odd/uncommon names in System32: `umppc19205.dll`, `CsXumd64_19205.dll`, `ScriptControl64_19205.dll`

## Malfind/VAD

- RWX private region @ 0x0000000000B70000 (size 0x1000) with raw x64 code. First bytes are `fc 48 83 e4 f0 e8 cc 00 00 00 41 51 41 50 52 48 31 d2 65 48 8b 52 60 ...`. A code stube in an RWX page inside spoolsv is a classic footprint indicator of injected shellcode.
- Numerous PAGE_EXECUTE_READWRITE regions and small thunk-like RWX stubs (e.g., at `0x7ffa...` ranges showing `48 FF 25 ...` RIP-relative jumps). Look like runtime trampolines/hooks.
- Very large RWX reservation shown by VAD (e.g. `0x7FFF0000-...`), further reinforcing memory manipulation well beyond normal spooler behavior.

## Notes

Evidence of code injection tino this spoolsv instance (RWX shellcode + trampolines), with capable networking and potential .NET hosting in-process.

In [25]:
# Handles / SIDs / privileges
!python "{VOL}" -f "{IMG}" windows.handles --pid 14536
!python "{VOL}" -f "{IMG}" windows.getsids --pid 14536
!python "{VOL}" -f "{IMG}" windows.privileges --pid 14536

Volatility 3 Framework 2.26.2

PID	Process	Offset	HandleValue	Type	GrantedAccess	Name

14536	spoolsv.exe	0x840e97fb1260	0x4	Event	0x1f0003	-
14536	spoolsv.exe	0xb70831f071d0	0x8	Key	0x9	MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
14536	spoolsv.exe	0x840ea0858f70	0xc	EtwRegistration	0x804	-
14536	spoolsv.exe	0x840e97fb14e0	0x10	Event	0x1f0003	-
14536	spoolsv.exe	0x840e9817c300	0x14	WaitCompletionPacket	0x1	-
14536	spoolsv.exe	0x840ea0858880	0x18	IoCompletion	0x1f0003	-
14536	spoolsv.exe	0x840e97f1d900	0x1c	TpWorkerFactory	0xf00ff	-
14536	spoolsv.exe	0x840ea4eba850	0x20	IRTimer	0x100002	-
14536	spoolsv.exe	0x840e9817b460	0x24	WaitCompletionPacket	0x1	-
14536	spoolsv.exe	0x840ea4eb9310	0x28	IRTimer	0x100002	-
14536	spoolsv.exe	0x840e9817b530	0x2c	WaitCompletionPacket	0x1	-
14536	spoolsv.exe	0x840ea0858790	0x30	EtwRegistration	0x804	-
14536	spoolsv.exe	0x840ea0858bf0	0x34	EtwRegistration	0x804	-
14536	spoolsv.exe	0x840ea08592f0	0x38	EtwRegistration	0x8


Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


Volatility 3 Framework 2.26.2

PID	Process	SID	Name

14536	spoolsv.exe	S-1-12-1-419028606-1323772725-945580434-2117921032	IanVeep
14536	spoolsv.exe	S-1-16-12288	High Mandatory Level
14536	spoolsv.exe	S-1-1-0	Everyone
14536	spoolsv.exe	S-1-5-32-544	Administrators
14536	spoolsv.exe	S-1-5-32-545	Users
14536	spoolsv.exe	S-1-5-4	Interactive
14536	spoolsv.exe	S-1-2-1	Console Logon (Users who are logged onto the physical console)
14536	spoolsv.exe	S-1-5-11	Authenticated Users
14536	spoolsv.exe	S-1-5-15	This Organization
14536	spoolsv.exe	S-1-5-5-0-40037722	Logon Session
14536	spoolsv.exe	S-1-2-0	Local (Users with the ability to log in locally)
14536	spoolsv.exe	S-1-5-64-36	-



Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


Volatility 3 Framework 2.26.2

PID	Process	Value	Privilege	Attributes	Description

14536	spoolsv.exe	2	SeCreateTokenPrivilege		Create a token object
14536	spoolsv.exe	3	SeAssignPrimaryTokenPrivilege		Replace a process-level token
14536	spoolsv.exe	4	SeLockMemoryPrivilege		Lock pages in memory
14536	spoolsv.exe	5	SeIncreaseQuotaPrivilege	Present	Increase quotas
14536	spoolsv.exe	6	SeMachineAccountPrivilege		Add workstations to the domain
14536	spoolsv.exe	7	SeTcbPrivilege		Act as part of the operating system
14536	spoolsv.exe	8	SeSecurityPrivilege	Present	Manage auditing and security log
14536	spoolsv.exe	9	SeTakeOwnershipPrivilege	Present	Take ownership of files/objects
14536	spoolsv.exe	10	SeLoadDriverPrivilege	Present	Load and unload device drivers
14536	spoolsv.exe	11	SeSystemProfilePrivilege	Present	Profile system performance
14536	spoolsv.exe	12	SeSystemtimePrivilege	Present	Change the system time
14536	spoolsv.exe	13	SeProfileSingleProcessPrivilege	Present	Profile a single proces


Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


# Analysis

## Handles

- File handle to Startup folder `\Device\HarddiskVolume3\Users\IanVeep\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Startup` is a strong hint of user-context persistence (dropper/LNK staging)
- AFD endpoints (`\Device\Afd\Endpoint`) indicate active/ready sockets
- GUID-named Section `F932B6C7-3A20-46A0-B8A0-8894AA421973` is uncommon

## SIDs

- Includes user SID IanVeep and interactive groups, plus admin groups; unusual for a “service” spooler instance that should be running under SYSTEM/Session 0. Indicates interactive/session-tied context.

## Privileges

- High-risk privileges present and enabled: `SeDebugPrivilege`, `SeImpersonatePrivilege`, plus many SYSTEM-level privileges (`SeTcbPrivilege`, `SeLoadDriverPrivilege`, etc.).
- Together with Session 2 lineage, this suggests the process is running with elevated/system-like capabilities in a user session (token abuse/impersonation or spawned as high-priv).

## Notes

Token/privilege profile and a startup-folder handle support post-exploitation objectives.

In [27]:
# Network corroboration ; recap from lab 2
!python "{VOL}" -f "{IMG}" windows.netscan | findstr /i "14536"

0x840e97632010	TCPv4	10.20.20.23	59865	35.86.58.69	80	CLOSED	14536	spoolsv.exe	2025-03-17 17:26:37.000000 UTC
0x840e99cb76e0	TCPv4	10.20.20.23	59879	35.86.58.69	80	CLOSED	14536	spoolsv.exe	2025-03-17 17:27:04.000000 UTC
0x840e99d64b60	TCPv4	10.20.20.23	59867	35.86.58.69	80	CLOSED	14536	spoolsv.exe	2025-03-17 17:26:39.000000 UTC
0x840e99d66520	TCPv4	10.20.20.23	59329	35.86.58.69	4444	CLOSED	14536	spoolsv.exe	2025-03-17 17:12:10.000000 UTC
0x840e9b199a70	TCPv4	10.20.20.23	59883	35.86.58.69	80	CLOSED	14536	spoolsv.exe	2025-03-17 17:27:06.000000 UTC
0x840e9b2dd010	TCPv4	10.20.20.23	59882	35.86.58.69	80	CLOSED	14536	spoolsv.exe	2025-03-17 17:27:06.000000 UTC
0x840e9bdeca20	TCPv4	10.20.20.23	59884	35.86.58.69	80	ESTABLISHED	14536	spoolsv.exe	2025-03-17 17:27:09.000000 UTC
0x840e9ce6c880	TCPv4	10.20.20.23	59881	35.86.58.69	80	CLOSED	14536	spoolsv.exe	2025-03-17 17:27:06.000000 UTC
0x840e9ea7da20	TCPv4	10.20.20.23	59875	35.86.58.69	80	CLOSED	14536	spoolsv.exe	2025-03-17 17:26:49.000000 UTC



Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:   23.33		Scanning Elf64Layer using PageMapScanner

Progress:    0.00		Scanning Elf64Layer using PageMapScanner

Progress:  100.00		Stacking attempts finished              

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:    0.00		Scanning layer_name using PdbSignatureScanner

Progress:  100.00		PDB scanning finished                        


# Recap from lab 2
- CLOSED 10.20.20.23:59329 -> 35.86.58.69:4444 at 17:12:10 (same minute this spoolsv starts).
- Multiple HTTP sessions to the same host:
  - CLOSED 10.20.20.23:59867/59865/59883/59882/59881 -> 35.86.58.69:80 around 17:26:37–17:27:06.
  - ESTABLISHED 10.20.20.23:59884 -> 35.86.58.69:80 at 17:27:09.
- Port 4444 and repeated hits to the same external IP are red flags for C2/handshake behavior.
- Legitimate print spooler traffic doesn’t originate as a child of PowerShell in a user session, and it typically doesn’t initiate outbound HTTP/4444 to the internet.
- The legit print spooler (spoolsv.exe PID 2904) is also present and only listens on RPC/ephemeral (LISTEN 0.0.0.0:49668), which is normal. Distinguish it from the rogue one by PID/parent/session and its outbound connections.

In [25]:
PID   = 14536                                     # target PID (e.g., rogue spoolsv.exe)
CARVE_START = 0x0000000000B70000                  # target VA (e.g., from malfind)
CARVE_SIZE  = 0x1000                              # bytes to carve from that VA

DUMP_DIR = CASE_DIR / "dumps" / f"pid_{PID}_memmap"
DUMP_DIR.mkdir(parents=True, exist_ok=True)

# Produce the dump file
#rc, so, se = run_cmd(f'python "{VOL}" -f "{IMG}" -o "{DUMP_DIR}" windows.memmap --pid {PID} --dump', live=False, timeout=3600)
if rc != 0:
    raise RuntimeError(f"volatility memmap --dump failed (rc={rc})")
dump_files = sorted(DUMP_DIR.glob("*.dmp"))
dump_path = dump_files[0]
print("Dump file:", dump_path)

# Get mapping table
#rc, map_out, map_err = run(f'python "{VOL}" -f "{IMG}" windows.memmap --pid {PID}', live=False, timeout=3600)
if rc != 0:
    # Some builds write table to stderr; try to use either
    if not map_out and map_err:
        map_out = map_err
    if not map_out:
        raise RuntimeError("Failed to obtain memmap table output")

# Parse rows and locate CARVE_START range to compute file offset
# Accept either header variants that include 'InFile' or 'Offset'
# Typical columns: Virtual Physical Size Offset InFile ...
line_re = re.compile(r'^\s*([0-9A-Fa-fx]+)\s+([0-9A-Fa-fx]+)\s+([0-9A-Fa-fx]+)\s+([0-9A-Fa-fx]+)\s+([0-9A-Fa-fx]+)')
rows = []
for line in map_out.splitlines():
    # Heuristic: pick lines that look like 5 hex columns
    m = line_re.match(line.strip())
    if not m:
        continue
    virtual = int(m.group(1), 16)
    physical = int(m.group(2), 16)
    size     = int(m.group(3), 16)
    offset   = int(m.group(4), 16)  # this may already be "InFile" in some builds
    infile   = int(m.group(5), 16)
    # Prefer the last column as "in-file" offset if it’s non-zero; else use the 4th
    file_off = infile if infile != 0 else offset
    rows.append((virtual, size, file_off))

if not rows:
    raise RuntimeError("Could not parse any memmap rows to compute file offsets.")

# Find the row whose VA range contains CARVE_START
file_offset = None
row_size = None
for va, sz, off in rows:
    if va <= CARVE_START < va + sz:
        file_offset = off + (CARVE_START - va)
        row_size = sz - (CARVE_START - va)
        break

if file_offset is None:
    # As a fallback, try exact-match rows (rare)
    for va, sz, off in rows:
        if va == CARVE_START:
            file_offset = off
            row_size = sz
            break

if file_offset is None:
    raise RuntimeError(f"Could not locate VA 0x{CARVE_START:x} in memmap table.")

# Carve bytes from the single dump
carve_len = min(CARVE_SIZE, row_size if row_size else CARVE_SIZE)
carve_path = DUMP_DIR / f"pid_{PID}_carve_0x{CARVE_START:x}.bin"
with open(dump_path, "rb") as f, open(carve_path, "wb") as g:
    f.seek(file_offset)
    g.write(f.read(carve_len))

print(f"Carved {carve_len} bytes from VA 0x{CARVE_START:x} @ file offset 0x{file_offset:x}")
print("Carve path:", carve_path)

Dump file: C:\Users\vboxuser\repos\CYBS-5014\lab3\output\dumps\pid_14536_memmap\pid.14536-1.dmp
Carved 4096 bytes from VA 0xb70000 @ file offset 0xd
Carve path: C:\Users\vboxuser\repos\CYBS-5014\lab3\output\dumps\pid_14536_memmap\pid_14536_carve_0xb70000.bin


# Analysis

- Saved full memmap dump and carved 0xB70000 shellcode page for static triage.  
- Carve method: translated VA in-file offset using `windows.memmap` table to precise extraction.

# Hash PID Dumps

In [26]:
artifacts = []
for p in sorted(DUMP_DIR.glob("**/*")):
    if p.is_file():
        artifacts.append({
            "file": str(p),
            "size": p.stat().st_size,
            "sha256": sha256(p),
            "mtime_utc": time.strftime("%Y-%m-%d %H:%M:%SZ", time.gmtime(p.stat().st_mtime))
        })

print(json.dumps(artifacts, indent=2))

[
  {
    "file": "C:\\Users\\vboxuser\\repos\\CYBS-5014\\lab3\\output\\dumps\\pid_14536_memmap\\pid.14536-1.dmp",
    "size": 1930354688,
    "sha256": "a3ffb85d91af9c0eaf06f96273acf01b2bed51c0ae3e00c91ea97b1bbe345b05",
    "mtime_utc": "2025-11-02 22:56:00Z"
  },
  {
    "file": "C:\\Users\\vboxuser\\repos\\CYBS-5014\\lab3\\output\\dumps\\pid_14536_memmap\\pid.14536.dmp",
    "size": 1930354688,
    "sha256": "a3ffb85d91af9c0eaf06f96273acf01b2bed51c0ae3e00c91ea97b1bbe345b05",
    "mtime_utc": "2025-11-02 22:49:14Z"
  },
  {
    "file": "C:\\Users\\vboxuser\\repos\\CYBS-5014\\lab3\\output\\dumps\\pid_14536_memmap\\pid_14536_carve_0xb70000.bin",
    "size": 4096,
    "sha256": "96e7a618d024c2e87e0c8ee4f3cd41e68f7c7b61898b44395d04e9b35334cace",
    "mtime_utc": "2025-11-02 23:06:42Z"
  }
]


# Static Analysis of Dumps

In [53]:
SAMPLES_DIR = DUMP_DIR
REPORT_DIR  = CASE_DIR / "static_reports"; REPORT_DIR.mkdir(parents=True, exist_ok=True)
results = []

for p in sorted(SAMPLES_DIR.glob("**/*")):
    if not p.is_file() or p.stat().st_size < 256: 
        continue
    rec = {"path": str(p), "size": p.stat().st_size, "sha256": sha256(p)}

    # Strings sample (first ~200 lines)
    with open(p, "rb") as f:
        data = f.read()
    rec["entropy_all"] = round(entropy(data[: min(len(data), 1<<20)]), 4)  # entropy of first 1MB
    s = ascii_strings(data[: min(len(data), 1<<20)], 6)[:200]
    rec["strings_head"] = s

    # PE details
    if pefile:
        try:
            pe = pefile.PE(str(p), fast_load=True)
            pe.parse_data_directories()
            ts = getattr(pe.FILE_HEADER, "TimeDateStamp", None)
            rec["pe_timestamp_utc"] = ts
            # Sections entropy
            secs = []
            for sec in pe.sections:
                raw = sec.get_data()
                secs.append({
                    "name": sec.Name.decode(errors="ignore").strip("\x00"),
                    "raw_size": sec.SizeOfRawData,
                    "virtual_size": sec.Misc_VirtualSize,
                    "entropy": round(entropy(raw), 4) if raw else None
                })
            rec["sections"] = secs
            # Imports (top few)
            imports = []
            if hasattr(pe, "DIRECTORY_ENTRY_IMPORT"):
                for entry in pe.DIRECTORY_ENTRY_IMPORT:
                    dll = entry.dll.decode(errors="ignore") if entry.dll else ""
                    funcs = [imp.name.decode(errors="ignore") if imp.name else f"ord{imp.ordinal}" for imp in entry.imports]
                    imports.append({"dll": dll, "sample": funcs[:15]})
            rec["imports"] = imports[:5]
            # Imphash
            try:
                rec["imphash"] = pe.get_imphash()
            except Exception:
                rec["imphash"] = None
        except Exception as e:
            rec["pe_error"] = str(e)

    results.append(rec)

# Save JSON
out_json = REPORT_DIR / "static_summary.json"
out_json.write_text(json.dumps(results, indent=2), encoding="utf-8")
print("Wrote:", out_json)

# Quick IOC skim from strings
ioc_candidates = {"hosts": set(), "ips": set(), "paths": set()}
import re
ip_re = re.compile(r"\b(?:\d{1,3}\.){3}\d{1,3}\b")
url_re = re.compile(r"(?:https?://)?[A-Za-z0-9\.\-]+\.[A-Za-z]{2,}(?:/[^\s\"']*)?")
path_re = re.compile(r"[A-Za-z]:\\[^\r\n\"']+")

for rec in results:
    for s in rec.get("strings_head", []):
        for m in ip_re.findall(s): ioc_candidates["ips"].add(m)
        for m in url_re.findall(s): ioc_candidates["hosts"].add(m)
        for m in path_re.findall(s): ioc_candidates["paths"].add(m)

print("IOC candidates (head of strings):")
print({k: list(sorted(v))[:20] for k,v in ioc_candidates.items()})

Wrote: C:\Users\vboxuser\repos\CYBS-5014\lab3\output\static_reports\static_summary.json
IOC candidates (head of strings):
{'hosts': ['Inter-Medium.woff/XtF34gk4tjq8W71Z24P1Tw8HhwlPVl_Uj77uYTSOcH9IDMSSNiqNWRCUw9FZ53gtNY8esjWXNPBq3NpMXyx0uG-ZfKHFhrQTBr5MOGbVvp1-woIXEUeE82W9q_OAQ9cojvJUpDZgpasB4d6jaGX6RHheH-bePrtt6b7yVED1ZHb6pPAQZtKIIWXngnzihC6lxppsFLqUStoTvzWoNoW9gwVJT25GQYYlLrAcWrwFzs_jJAHf2', 'cmd.exe', 'mswsock.dll', 'spoolsv.exe'], 'ips': ['35.86.58.69', '8.0.422.05'], 'paths': ['C:\\Program Files', 'C:\\Program Files (x86)', 'C:\\Program Files (x86)\\Common Files', 'C:\\Program Files\\Common Files', 'C:\\Program Files\\OpenLogic\\jdk-8.0.422.05-hotspot\\bin;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\Windows\\System32\\OpenSSH\\;C:\\gradle-6.8.3\\bin;C:\\Users\\IanVeep\\AppData\\Local\\Microsoft\\WindowsApps', 'C:\\ProgramData', 'C:\\Users\\IanVeep', 'C:\\Users\\IanVeep\\AppData\\Local', 'C:\\Users\\IanVeep\\AppD

# Analysis
- Artifacts analyzed (from static_summary.json):
  - pid.14536.dmp (spoolsv.exe full memmap dump)
    - SHA-256: a3ffb85d91af9c0eaf06f96273acf01b2bed51c0ae3e00c91ea97b1bbe345b05
    - Entropy (first 1 MB): 2.1164 (mixed code/data; not purely packed)
    - PE parse: not a valid PE (DOS Header magic not found.) which is expected for a raw process dump.

- pid_14536_carve_0xb70000.bin (RWX page carve)
  - SHA-256: 96e7a618d024c2e87e0c8ee4f3cd41e68f7c7b61898b44395d04e9b35334cace
  - Entropy: 0.3861 (structured shellcode/zeroes)
  - PE parse: not a PE - expected for a shellcode page.

Clear network/C2 evidence inside pid.14536.dmp strings:
- Direct IP C2: 35.86.58.69 (also seen in netscan).
- HTTP over non-standard port: explicit request and response captured in strings:
  - GET /fonts/static/Inter-Medium.woff/<long token> HTTP/1.1
  - http://35.86.58.69:4444/fonts/static/Inter-Medium.woff/<long token>
  - HTTP/1.1 200 OK, Transfer-Encoding: chunked, Cache-Control: no-store, no-cache, must-revalidate, Date: Mon, 17 Mar 2025 17:12:11 GMT, Content-Type: application/octet-stream. Strong indicator of HTTP C2 masquerading as font delivery on port 4444, consistent with the runtime netscan timeline.

Defense/AV-evasion & LOLBAS hints (function imports/strings):
- AmsiInitialize, AmsiScanBuffer, AmsiScanString (AMSI touchpoints).
- WldpQueryDynamicCodeTrust, WldpIsClassInApprovedList (WDAC/Code-Integrity related).
- winhttp, wininet, ole32/oleaut32 present in strings (network + COM usage).
- Multiple standard environment strings (PATH, ComSpec=C:\Windows\system32\cmd.exe, USERNAME=IanVeep), AzureAD user domain, and system folders — useful for situational context, not IOCs by themselves.

Noise & false positives to ignore:
- "8.0.422.05" (caught by the IP regex) is a JDK version, not an IP.
- "cmd.exe", "mswsock.dll", "spoolsv.exe" appeared in the “hosts” bucket due to a loose pattern - not hosts.
- Generic Windows and user profile paths are normal context; keep only paths tied to malicious behavior.

Takeaways for YARA/hunting (seed ideas):
- Byte/ASCII markers from the carved RWX page (low entropy), use as shellcode anchors.
- HTTP path pattern: /fonts/static/Inter-Medium.woff/ followed by a long base64-ish token.
- Presence of AMSI/WDLP symbols combined with WinHTTP/WinINet in a process that shouldn’t be talking HTTP (spooler).


# Summary

## Overview
- spoolsv.exe (PID 14536) is an additional suspicious process: it runs in Session 2 as a child of interactive PowerShell, not the service tree, and contains RWX shellcode at 0x0000000000B70000.
- From static strings in the full process dump, we see direct-IP HTTP C2 to 35.86.58.69, disguising traffic as a font download:
`GET /fonts/static/Inter-Medium.woff/<token> HTTP/1.1` on port 4444, with `HTTP/1.1 200 OK, Transfer-Encoding: chunked`, and `Cache-Control: no-store, no-cache, must-revalidate` are fully consistent with the netscan timeline.
- The dump also contains AMSI/WDLP symbols (e.g., AmsiScanBuffer, WldpQueryDynamicCodeTrust) and WinHTTP/WinINet markers, supporting in-process HTTP C2 and likely defense-aware behavior.
- The process chain remains: PSEXESVC.exe -> excel-update.exe -> cmd.exe -> powershell.exe -> spoolsv.exe, with excel-update.exe earlier beaconing to 35.86.58.69:4443 and spoolsv.exe later communicating on 4444/80 — a single C2 infrastructure across stages.

## Next steps:

- Contain the host; terminate the rogue spoolsv.exe and remove Startup-folder persistence.
- Block 35.86.58.69 and hunt across the estate for HTTP requests to /fonts/static/Inter-Medium.woff/ on 80/4443/4444.
- Scope for C:\Windows\excel-update.exe and look-alikes; review service/task creation around 17:01–17:27 UTC.
- Use the IOC pack above (plus the shellcode page hash as a memory IOC) to drive detection, and reimage where compromise depth warrants.